<a href="https://colab.research.google.com/github/Neil-Sachdeva/RoboGAN/blob/master/RoboGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Data Formatting**

In [0]:
import numpy as np
import cv2
import gc

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

import os
os.chdir('gdrive/My Drive/')

In [0]:
# Change datapaths and upload path

clip = cv2.VideoCapture("/content/drive/My Drive/data18/Training/Dataset1/Video.avi")
clip_L = cv2.VideoCapture("/content/drive/My Drive/data18/Training/Dataset1/Left_Instrument_Segmentation.avi")
clip_R = cv2.VideoCapture("/content/drive/My Drive/data18/Training/Dataset1/Right_Instrument_Segmentation.avi")

i = 0

path = "/content/drive/My Drive/pytorch-CycleGAN-and-pix2pix/datasets/ML_Data/%s/Robo_In_%4d.jpeg"
while clip.isOpened() and clip_L.isOpened() and clip_R.isOpened():
    ret, frame = clip.read()
    ret_L, frame_L = clip_L.read()
    ret_R, frame_R = clip_R.read()
    if ret and ret_L and ret_R:
        frame_LR = np.sum((frame_L, frame_R), axis=0)
        frame_cat = np.concatenate((frame, frame_LR), axis=1)
        cv2.imwrite(path % ("val" if i > 885 else "train",i), frame_cat)
        i += 1
        del frame_LR
        del frame_cat
        if i % 50 == 0:
            print("finished %d images"%i)
    else:
        break
    del ret
    del ret_L
    del ret_R
    del frame
    del frame_L
    del frame_R
    gc.collect()

## **Model Training**

In [0]:
# Must run this cell prior to any model training, testing, or timing
# Clones repository that hosts pix2pix model used in this work
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [0]:
get_ipython().system_raw('python3 -m pip3 install visdom')
get_ipython().system_raw('python3 -m visdom.server -port 8097 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('ssh -o ServerAliveInterval=60 -o StrictHostKeyChecking=no -R MPCR_Visdom:80:localhost:8097 serveo.net &')
print('Visdom view: {}'.format('https://MPCR_Visdom.serveo.net/'))

import shutil
import numpy as np
import time
import tempfile
from six.moves import urllib
import matplotlib.pyplot as plt
from glob import glob
!pip3 install scipy==1.1.0

shutil.move((data_dir), (git_dir + '/datasets/'))

In [0]:
os.chdir(git_dir)
!pip3 install -r requirements.txt

In [0]:
!python3 train.py --dataroot ./datasets/ML_Data --name Robo_Surgery --model pix2pix
!python3 test.py --dataroot ./datasets/ML_Data --name Robo_Surgery --model pix2pix

## **Model Testing**

In [0]:
# Change paths and directory

data_path = '/content/drive/My Drive/pytorch-CycleGAN-and-pix2pix/datasets/ML_Data'
model_path = '/content/drive/My Drive/pytorch-CycleGAN-and-pix2pix/checkpoints/Robo_Surgery'

os.chdir('drive/My Drive/pytorch-CycleGAN-and-pix2pix/')

In [0]:
!pip install dominate
!pip install scipy==1.1.0

In [0]:
# Change --dataroot and --name
!python test.py --dataroot '/content/drive/My Drive/pytorch-CycleGAN-and-pix2pix/datasets/ML_Data' --direction AtoB --name Robo_Surgery --model pix2pix

## **Pixel Difference**

In [0]:
import matplotlib.pyplot as plt

def scale(x):
  
  return (x-np.min(x))/(np.max(x))

def plot(x):
    fig, ax = plt.subplots()
    im = ax.imshow(scale(x), cmap = 'gray')
    ax.axis('off')
    plt.show()

In [0]:
# Change data paths

path = "/content/drive/My Drive/pytorch-CycleGAN-and-pix2pix/checkpoints/Robo_Surgery/web/images/"
fake1 = cv2.imread(path + "epoch001_fake_B.png")
real1 = cv2.imread(path + "epoch001_real_B.png")
fake200 = cv2.imread(path + "epoch200_fake_B.png")
real200 = cv2.imread(path + "epoch200_real_B.png")
diff1 = real1-fake1
diff200 = real200-fake200

In [0]:
plot(fake1)
plot(real1)
plot(diff1)

In [0]:
plot(fake200)
plot(real200)
plot(diff200)

In [0]:
plt.hist(diff1.flatten(),50)

In [0]:
plt.hist(diff200.flatten(),50)